In [0]:
!pip install openai==0.28

In [0]:
dbutils.library.restartPython()

In [0]:
import openai
openai.api_type = "azure"
openai.api_base = "https://oai-dk.openai.azure.com/"
openai.api_version = "2023-12-01-preview"
openai.api_key = dbutils.secrets.get('akvdesafiokinea','azure-oai-dk')

In [0]:
#Função para Gerar Restrição
def gerar_restricao(regra,dic):
    message_text = [
                {"role":"system",
                "content":
                    """
                    Você receberá o texto em relação a uma regra e um dicionário de valores em torno daquela regra. Sua resposta deve ser uma função de restrição no estilo scipy no formato:
                    def restricao(x):
                        return x
                    e o tipo de restrição "ineq" para inequações e "eq" para equações
                """
                },
                {"role":"user","content":f"Texto para a regra: {regra}, JSON com Resultados: {dic}"}
               ]
    
    completion = openai.ChatCompletion.create(
        engine="gpt35turbo16k",
        messages = message_text,
        temperature=0,
        max_tokens=1200,
        top_p=0.80,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None)

    return completion.to_dict()['choices'][0]['message']['content']

In [0]:
ex_dic= {'ativo': 'UNEG11', 'pior_rating_permitido': 2, 'rating do ativo': 6, 'posicao_ativo': 118787475.0} #A partir do Parser
ex_regra = "Vedação - Ativos de Crédito Privado c/ Rating Diferente de A" #A partir de nxenq_regras.Descricao

resposta = gerar_restricao(ex_regra,ex_dic)
print(resposta)


In [0]:
#acho que da pra melhorar o prompt

ex_dic = {
    'ativo': 'UNEG11',
    'pior_rating_permitido': 2, 
    'rating_do_ativo': 6,
    'posicao_ativo': 118787475.0,
    'descricao_regra': 'Vedação - Ativos de Crédito Privado c/ Rating Diferente de A'
}

ex_regra = """
Descrição da Regra:
1. Tipo de ativo: Crédito Privado.
2. A regra proíbe ativos com rating diferente de A.
3. O rating do ativo fornecido é 6, o que viola a regra.
4. O ativo em questão é 'UNEG11', e sua posição é 118787475.0.
"""
resposta = gerar_restricao(ex_regra,ex_dic)
print(resposta)


In [0]:
#podemos pensar também em como fazer um prompt que permita receber varias instruções e retorne restrições diferentes para cada uma delas 
def gerar_restricao(mensagem_regra, dados_ativos):
    """
    Função que gera restrições com base na regra e dados dos ativos.
    
    Parâmetros:
    - mensagem_regra (str): Descrição da regra a ser aplicada.
    - dados_ativos (dict): Dicionário com os dados do ativo e da regra.
    
    Retorno:
    - (str): Mensagem que descreve a restrição gerada.
    """
    ativo = dados_ativos['ativo']
    rating_ativo = dados_ativos['rating_do_ativo']
    rating_permitido = dados_ativos['pior_rating_permitido']
    descricao_regra = dados_ativos.get('descricao_regra', 'Regra não especificada')

    if rating_ativo > rating_permitido:
        resposta = f"O ativo '{ativo}' não cumpre a regra '{descricao_regra}', pois seu rating ({rating_ativo}) excede o limite permitido ({rating_permitido})."
    else:
        resposta = f"O ativo '{ativo}' está em conformidade com a regra '{descricao_regra}'."

    return resposta